In [158]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## my credentials

In [249]:
CLIENT_ID = 'VEOZPGP0Z3T2HVH3HE51A03YJM5BGH4HCWBMT4CSSYMHAVEY' # your Foursquare ID
CLIENT_SECRET = 'DZ1F3ILNQU15JLPWSNRMED01MS31GSAEOIPTSFIOJDBNOC3O' # your Foursquare Secret
VERSION = '20200830'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VEOZPGP0Z3T2HVH3HE51A03YJM5BGH4HCWBMT4CSSYMHAVEY
CLIENT_SECRET:DZ1F3ILNQU15JLPWSNRMED01MS31GSAEOIPTSFIOJDBNOC3O


# get location

In [257]:
address = '225 North Avenue NW, Atlanta, GA '

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

33.7722178 -84.3944354


In [355]:
radius = 1000
LIMIT = 100

# explore near location

In [356]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=VEOZPGP0Z3T2HVH3HE51A03YJM5BGH4HCWBMT4CSSYMHAVEY&client_secret=DZ1F3ILNQU15JLPWSNRMED01MS31GSAEOIPTSFIOJDBNOC3O&ll=33.7722178,-84.3944354&v=20200830&radius=1000&limit=100'

In [358]:
import requests
results = requests.get(url).json()
'There are {} venues arround Georgia Tech.'.format(len(results['response']['groups'][0]['items']))

'There are 77 venues arround Georgia Tech.'

In [360]:
items = results['response']['groups'][0]['items']

In [367]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.delivery.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.dropna()
# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.dropna()

<ipython-input-367-94222343b825>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(items) # flatten JSON


,name,categories,id,url,name,prefix,sizes,name,id
4,Highland Bakery,Bakery,1490525,https://www.grubhub.com/restaurant/highland-ba...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,512e4185e0e26a825ba3c825
13,Atwoods Pizza Cafe,Pizza Place,2213140,https://www.grubhub.com/restaurant/atwoods-piz...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,57156db6498e844a5b3b8837
16,Momonoki,Japanese Restaurant,975665,https://www.grubhub.com/restaurant/momonoki-95...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,5b27d937b3c961002c2af37c
26,Negril Village ATL,Caribbean Restaurant,1446896,https://www.grubhub.com/restaurant/negril-vill...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,52a0d3d411d2bcc3ecda9aa6
42,Land Of A Thousand Hills Coffee,Coffee Shop,1760197,https://www.grubhub.com/restaurant/land-of-a-t...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,592851120ff4f9137e7da9ec
43,Baraonda Cafe Italiano,Italian Restaurant,1490828,https://www.grubhub.com/restaurant/baraonda-71...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,49e3c071f964a520c6621fe3
50,Tropical Smoothie Cafe,Smoothie Shop,1170064,https://www.grubhub.com/restaurant/tropical-sm...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,594d406231fd145b63bdc58b
51,The Halal Guys,North Indian Restaurant,1176020,https://www.grubhub.com/restaurant/the-halal-g...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,5bb30c5c67f62b00392d0a59
55,Satto Thai & Sushi Bar,Thai Restaurant,277551,https://www.grubhub.com/restaurant/satto-thai-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,510aad23e4b0bfb25831b9dc
58,Wingnuts,American Restaurant,259023,https://www.grubhub.com/restaurant/wingnuts-76...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,4bf0b8e8f45ec928072f943c
